In [1]:
# remove warning messages

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from sklearn import preprocessing
from IPython.display import display, HTML

df=pd.read_csv('malware_BinaryImbalanced.csv')


# keep 19 columns and remove others which are not appropriate to be included in this classification task
df=df.drop(['hash','millisecond','policy','state','cached_hole_size','free_area_cache'
             ,'mm_users','end_data','last_interval','min_flt','maj_flt','fs_excl_counter'
             ,'lock','utime','stime','gtime','cgtime'],axis=1)

display(HTML(df.head(10).to_html()))

# check again whether there are missing values
print('ColumnName, DataType, MissingValues')
for i in df.columns:
    print(i, ',', df[i].dtype,',',df[i].isnull().any())

# encode labels
y = df['classification'] # define label as nominal values
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y) # encode nominal labels to integers #####################################
df['classification'] = y_encoded

# print out and display dataframe as tables in HTML
display(HTML(df.head(10).to_html()))

print('Column Datatypes:\n',df.dtypes)

# convert all nominal variables to binary variables
df_num=df.copy(deep=True) 
# create new binary columns
df_dummies=pd.get_dummies(df_num[['os']])
# add them to dataframe
df_num=df_num.join(df_dummies)
# drop original columns
df_num=df_num.drop('os',axis=1)

display(HTML(df_num.head(10).to_html()))

# drop extra binary columns, since we only need N-1 binary columns
df_num=df_num.drop('os_CentOS', axis=1)
display('df_num:',HTML(df_num.head(10).to_html()))


,classification,os,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw
0,benign,Ubuntu,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
1,benign,CentOS,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
2,benign,Ubuntu,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
3,benign,CentOS,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
4,benign,Mac,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
5,benign,Windows,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
6,benign,Windows,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
7,benign,CentOS,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
8,benign,Ubuntu,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
9,benign,Mac,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0


ColumnName, DataType, MissingValues
classification , object , False
os , object , False
usage_counter , int64 , False
prio , int64 , False
static_prio , int64 , False
normal_prio , int64 , False
vm_pgoff , int64 , False
vm_truncate_count , int64 , False
task_size , int64 , False
map_count , int64 , False
hiwater_rss , int64 , False
total_vm , int64 , False
shared_vm , int64 , False
exec_vm , int64 , False
reserved_vm , int64 , False
nr_ptes , int64 , False
nvcsw , int64 , False
nivcsw , int64 , False
signal_nvcsw , int64 , False


,classification,os,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw
0,0,Ubuntu,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
1,0,CentOS,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
2,0,Ubuntu,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
3,0,CentOS,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
4,0,Mac,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
5,0,Windows,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
6,0,Windows,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
7,0,CentOS,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
8,0,Ubuntu,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0
9,0,Mac,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0


Column Datatypes:
 classification        int32
os                   object
usage_counter         int64
prio                  int64
static_prio           int64
normal_prio           int64
vm_pgoff              int64
vm_truncate_count     int64
task_size             int64
map_count             int64
hiwater_rss           int64
total_vm              int64
shared_vm             int64
exec_vm               int64
reserved_vm           int64
nr_ptes               int64
nvcsw                 int64
nivcsw                int64
signal_nvcsw          int64
dtype: object


,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_CentOS,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,0,1,0
1,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,1,0,0,0,0
2,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,0,1,0
3,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,1,0,0,0,0
4,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,1,0,0
5,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,0,0,1
6,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,0,0,1
7,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,1,0,0,0,0
8,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,0,1,0
9,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,1,0,0


'df_num:'

,classification,usage_counter,prio,static_prio,normal_prio,vm_pgoff,vm_truncate_count,task_size,map_count,hiwater_rss,total_vm,shared_vm,exec_vm,reserved_vm,nr_ptes,nvcsw,nivcsw,signal_nvcsw,os_Debian,os_Mac,os_Ubuntu,os_Windows
0,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,1,0
1,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,0,0
2,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,1,0
3,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,0,0
4,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,1,0,0
5,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,0,1
6,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,0,1
7,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,0,0
8,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,0,1,0
9,0,0,3069403136,16447,0,0,14739,0,7903,0,88,120,120,80,0,349169,0,0,0,1,0,0


In [16]:
#Logistic regression
#First, we have to find the best parameters accroding to accuracy
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
# Define the parameter combinations to test
penalties = ['l1', 'l2']
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'saga']
max_iters = [100, 150, 200]
Cs = [0.1, 1.0, 10.0]
fit_intercepts = [True, False]

x=df_num.drop('classification',axis=1)
y=df_num['classification']

# Assuming x and y are already defined as per your code
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

best_accuracy = 0
best_params = {}

for penalty in penalties:
    for solver in solvers:
        # Check if the combination is valid
        if (penalty == 'l1' and solver not in ['liblinear', 'saga']) or \
           (penalty == 'elasticnet' and solver not in ['saga']) or \
           (penalty == 'l2' and solver == 'lbfgs'):
            continue

        for max_iter in max_iters:
            if solver == 'liblinear' and penalty == 'l1':
                continue
            
            for C in Cs:
                for fit_intercept in fit_intercepts:
                    clf = LogisticRegression(
                        penalty=penalty,
                        solver=solver,
                        max_iter=max_iter,
                        C=C,
                        fit_intercept=fit_intercept,
                        multi_class='ovr', 
                        random_state=42
                    )
                    clf = clf.fit(x_train, y_train)
                    y_pred = clf.predict(x_test)
                    acc = accuracy_score(y_test, y_pred)

                    # Print accuracy for each model
                    print(f"Parameters: penalty={penalty}, solver={solver}, max_iter={max_iter}, C={C}, fit_intercept={fit_intercept}")
                    print(f"Accuracy: {acc}\n")
                    
                    if acc > best_accuracy:
                        best_accuracy = acc
                        best_params = {
                            'penalty': penalty,
                            'solver': solver,
                            'max_iter': max_iter,
                            'C': C,
                            'fit_intercept': fit_intercept
                        }

print("Best Parameters:", best_params, "Best Accuracy:", best_accuracy)

Parameters: penalty=l1, solver=saga, max_iter=100, C=0.1, fit_intercept=True
Accuracy: 0.8978

Parameters: penalty=l1, solver=saga, max_iter=100, C=0.1, fit_intercept=False
Accuracy: 0.8978

Parameters: penalty=l1, solver=saga, max_iter=100, C=1.0, fit_intercept=True
Accuracy: 0.8978

Parameters: penalty=l1, solver=saga, max_iter=100, C=1.0, fit_intercept=False
Accuracy: 0.8978

Parameters: penalty=l1, solver=saga, max_iter=100, C=10.0, fit_intercept=True
Accuracy: 0.8978

Parameters: penalty=l1, solver=saga, max_iter=100, C=10.0, fit_intercept=False
Accuracy: 0.8978

Parameters: penalty=l1, solver=saga, max_iter=150, C=0.1, fit_intercept=True
Accuracy: 0.8978

Parameters: penalty=l1, solver=saga, max_iter=150, C=0.1, fit_intercept=False
Accuracy: 0.8978

Parameters: penalty=l1, solver=saga, max_iter=150, C=1.0, fit_intercept=True
Accuracy: 0.8978

Parameters: penalty=l1, solver=saga, max_iter=150, C=1.0, fit_intercept=False
Accuracy: 0.8978

Parameters: penalty=l1, solver=saga, max_it

In [27]:
# Logistic regression
import pandas as pd
import numpy as np
from sklearn import metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix as cm

x=df_num.drop('classification',axis=1)
y=df_num['classification']

# by hold-out evaluation
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
clf=LogisticRegression(penalty='l2',solver='newton-cg',
                       max_iter=150, multi_class='ovr', C= 10.0, fit_intercept= False)
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)

confM = cm(y_test, y_pred)
print(confM)

acc=accuracy_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred, average='micro')
auc = roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1], average='macro')
print('By hold-out evaluation: acc = ',acc, ', F1 = ',test_f1,', AUC = ',auc )

[[21856   577]
 [ 1915   652]]
By hold-out evaluation: acc =  0.90032 , F1 =  0.90032 , AUC =  0.8994358320446267


In [18]:
# SVM
# Find the best parameters accroding to accuracy

from sklearn.svm import SVC

# Define the parameter combinations to test
Cs = [0.1, 1.0, 10.0]
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
max_iters = [100, 200, 300]
cache_sizes = [200, 400]

# Assuming x and y are already defined as per your code
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

best_accuracy = 0
best_params = {}

for C in Cs:
    for kernel in kernels:
        for max_iter in max_iters:
            for cache_size in cache_sizes:
                clf = SVC(
                    C=C,
                    kernel=kernel,
                    max_iter=max_iter,
                    cache_size=cache_size,
                    random_state=42  
                )
                clf.fit(x_train, y_train)
                y_pred = clf.predict(x_test)
                acc = accuracy_score(y_test, y_pred)

                # Print accuracy for each model
                print(f"Parameters: C={C}, kernel={kernel}, max_iter={max_iter}, cache_size={cache_size}")
                print(f"Accuracy: {acc}\n")

                if acc > best_accuracy:
                    best_accuracy = acc
                    best_params = {
                        'C': C,
                        'kernel': kernel,
                        'max_iter': max_iter,
                        'cache_size': cache_size
                    }

print("Best Parameters:", best_params, "Best Accuracy:", best_accuracy)


Parameters: C=0.1, kernel=linear, max_iter=100, cache_size=200
Accuracy: 0.37276

Parameters: C=0.1, kernel=linear, max_iter=100, cache_size=400
Accuracy: 0.37276

Parameters: C=0.1, kernel=linear, max_iter=200, cache_size=200
Accuracy: 0.35236

Parameters: C=0.1, kernel=linear, max_iter=200, cache_size=400
Accuracy: 0.35236

Parameters: C=0.1, kernel=linear, max_iter=300, cache_size=200
Accuracy: 0.34052

Parameters: C=0.1, kernel=linear, max_iter=300, cache_size=400
Accuracy: 0.34052

Parameters: C=0.1, kernel=poly, max_iter=100, cache_size=200
Accuracy: 0.51396

Parameters: C=0.1, kernel=poly, max_iter=100, cache_size=400
Accuracy: 0.51396

Parameters: C=0.1, kernel=poly, max_iter=200, cache_size=200
Accuracy: 0.5036

Parameters: C=0.1, kernel=poly, max_iter=200, cache_size=400
Accuracy: 0.5036

Parameters: C=0.1, kernel=poly, max_iter=300, cache_size=200
Accuracy: 0.5036

Parameters: C=0.1, kernel=poly, max_iter=300, cache_size=400
Accuracy: 0.5036

Parameters: C=0.1, kernel=rbf, m

In [26]:
#SVM
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

# Train the SVM classifier with a polynomial kernel
clf = SVC(kernel='sigmoid', C=0.1, max_iter=200, cache_size=200, probability=True)  # Set probability=True for predict_proba
clf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(x_test)

# Calculate accuracy
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro')

# Calculate macro-AUC using a one-vs-rest (OvR) strategy
n_classes = len(np.unique(y_test))
y_pred_proba = clf.predict_proba(x_test)
macro_auc = 0
for i in range(n_classes):
    y_true_i = (y_test == i).astype(int)
    auc_i = roc_auc_score(y_true_i, y_pred_proba[:, i])
    macro_auc += auc_i
macro_auc /= n_classes

print('By hold-out evaluation: acc = ', acc, ', F1 = ', f1, ', AUC = ', macro_auc)

By hold-out evaluation: acc =  0.89932 , F1 =  0.89932 , AUC =  0.6849705240586933


In [20]:
#Random Forest
# Find the best parameters accroding to accuracy

from sklearn.ensemble import RandomForestClassifier

# Define the parameter combinations to test
n_estimators = [100, 150, 200]
max_depths = [None, 10, 20]
min_samples_splits = [2, 5, 10]
min_samples_leafs = [1, 3, 5]

# Assuming x and y are already defined as per your code
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

best_accuracy = 0
best_params = {}

for n_estimator in n_estimators:
    for max_depth in max_depths:
        for min_samples_split in min_samples_splits:
            for min_samples_leaf in min_samples_leafs:
                clf = RandomForestClassifier(
                    n_estimators=n_estimator,
                    max_depth=max_depth,
                    min_samples_split=min_samples_split,
                    min_samples_leaf=min_samples_leaf,
                    random_state=42  
                )
                clf.fit(x_train, y_train)
                y_pred = clf.predict(x_test)
                acc = accuracy_score(y_test, y_pred)

                # Print accuracy for each model
                print(f"Parameters: n_estimators={n_estimator}, max_depth={max_depth}, min_samples_split={min_samples_split}, min_samples_leaf={min_samples_leaf}")
                print(f"Accuracy: {acc}\n")

                if acc > best_accuracy:
                    best_accuracy = acc
                    best_params = {
                        'n_estimators': n_estimator,
                        'max_depth': max_depth,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf
                    }

print("Best Parameters:", best_params, "Best Accuracy:", best_accuracy)

Parameters: n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1
Accuracy: 1.0

Parameters: n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=3
Accuracy: 1.0

Parameters: n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=5
Accuracy: 1.0

Parameters: n_estimators=100, max_depth=None, min_samples_split=5, min_samples_leaf=1
Accuracy: 1.0

Parameters: n_estimators=100, max_depth=None, min_samples_split=5, min_samples_leaf=3
Accuracy: 1.0

Parameters: n_estimators=100, max_depth=None, min_samples_split=5, min_samples_leaf=5
Accuracy: 1.0

Parameters: n_estimators=100, max_depth=None, min_samples_split=10, min_samples_leaf=1
Accuracy: 1.0

Parameters: n_estimators=100, max_depth=None, min_samples_split=10, min_samples_leaf=3
Accuracy: 1.0

Parameters: n_estimators=100, max_depth=None, min_samples_split=10, min_samples_leaf=5
Accuracy: 1.0

Parameters: n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=1
Accu

In [21]:
#Bagging - RandomForest

from sklearn import metrics 
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, precision_score, accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

x=df_num.drop('classification',axis=1)
y=df_num['classification']

# by hold-out evaluation
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
clf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1)
clf = clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)

acc=accuracy_score(y_test, y_pred)
f1_score = f1_score(y_test, y_pred, average='micro')
roc_auc = roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1], average='macro')

print('By hold-out evaluation: acc = ',acc, ', F1 = ',f1_score,', AUC = ',auc)

By hold-out evaluation: acc =  1.0 , F1 =  1.0 , AUC =  0.8214122677176305


In [22]:
# Bagging (Naïve Bayes)
# Find the best parameters accroding to accuracy
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB

# Define the parameter combinations to test
n_estimators = [10, 100, 200]
max_samples = [1.0, 0.5, 0.1]
max_features = [1.0, 0.5, 0.1]
bootstrap = [True, False]

# Assuming x and y are already defined as per your code
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

best_accuracy = 0
best_params = {}

for n_estimator in n_estimators:
    for sample_ratio in max_samples:
        for feature_ratio in max_features:
            for use_bootstrap in bootstrap:
                clf = BaggingClassifier(
                    base_estimator=GaussianNB(),
                    n_estimators=n_estimator,
                    max_samples=sample_ratio,
                    max_features=feature_ratio,
                    bootstrap=use_bootstrap,
                    random_state=42 
                )
                clf.fit(x_train, y_train)
                y_pred = clf.predict(x_test)
                acc = accuracy_score(y_test, y_pred)

                # Print accuracy for each model
                print(f"Parameters: n_estimators={n_estimator}, max_samples={sample_ratio}, max_features={feature_ratio}, bootstrap={use_bootstrap}")
                print(f"Accuracy: {acc}\n")

                if acc > best_accuracy:
                    best_accuracy = acc
                    best_params = {
                        'n_estimators': n_estimator,
                        'max_samples': sample_ratio,
                        'max_features': feature_ratio,
                        'bootstrap': use_bootstrap
                    }

print("Best Parameters:", best_params, "Best Accuracy:", best_accuracy)

Parameters: n_estimators=10, max_samples=1.0, max_features=1.0, bootstrap=True
Accuracy: 0.76356

Parameters: n_estimators=10, max_samples=1.0, max_features=1.0, bootstrap=False
Accuracy: 0.76348

Parameters: n_estimators=10, max_samples=1.0, max_features=0.5, bootstrap=True
Accuracy: 0.80196

Parameters: n_estimators=10, max_samples=1.0, max_features=0.5, bootstrap=False
Accuracy: 0.80212

Parameters: n_estimators=10, max_samples=1.0, max_features=0.1, bootstrap=True
Accuracy: 0.90012

Parameters: n_estimators=10, max_samples=1.0, max_features=0.1, bootstrap=False
Accuracy: 0.90012

Parameters: n_estimators=10, max_samples=0.5, max_features=1.0, bootstrap=True
Accuracy: 0.76356

Parameters: n_estimators=10, max_samples=0.5, max_features=1.0, bootstrap=False
Accuracy: 0.76348

Parameters: n_estimators=10, max_samples=0.5, max_features=0.5, bootstrap=True
Accuracy: 0.80176

Parameters: n_estimators=10, max_samples=0.5, max_features=0.5, bootstrap=False
Accuracy: 0.80216

Parameters: n_e

In [23]:
# Bagging
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, roc_auc_score, accuracy_score

# by Hold-out Evaluation
x=df_num.drop('classification',axis=1)
y=df_num['classification']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
clf = GaussianNB()
bag = BaggingClassifier(clf, n_estimators=10, max_samples=1.0, max_features= 0.1, bootstrap= True)
bag.fit(x_train, y_train)
y_pred = bag.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
roc_auc = roc_auc_score(y_test, bag.predict_proba(x_test)[:, 1], average='macro')

print("Bagging using GaussianNB Accuracy by Hold-out Evaluation: acc = ",accuracy, ', F1 = ',f1,', AUC = ',roc_auc)

Bagging using GaussianNB Accuracy by Hold-out Evaluation: acc =  0.89824 , F1 =  0.47319622387053273 , AUC =  0.8521336378561682


In [24]:
# GradientBoostingClassifier
# Find the best parameters accroding to accuracy

from sklearn.ensemble import GradientBoostingClassifier

# Define the parameter combinations to test
learning_rates = [0.1, 0.01, 0.001]
n_estimators = [100, 200, 300]
subsample = [1.0, 0.8, 0.5]
max_depths = [3, 4, 5]

# Assuming x and y are already defined as per your code
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

best_accuracy = 0
best_params = {}

for learning_rate in learning_rates:
    for n_estimator in n_estimators:
        for sample_ratio in subsample:
            for max_depth in max_depths:
                clf = GradientBoostingClassifier(
                    learning_rate=learning_rate,
                    n_estimators=n_estimator,
                    subsample=sample_ratio,
                    max_depth=max_depth,
                    random_state=42 
                )
                clf.fit(x_train, y_train)
                y_pred = clf.predict(x_test)
                acc = accuracy_score(y_test, y_pred)

                # Print accuracy for each model
                print(f"Parameters: learning_rate={learning_rate}, n_estimators={n_estimator}, subsample={sample_ratio}, max_depth={max_depth}")
                print(f"Accuracy: {acc}\n")

                if acc > best_accuracy:
                    best_accuracy = acc
                    best_params = {
                        'learning_rate': learning_rate,
                        'n_estimators': n_estimator,
                        'subsample': sample_ratio,
                        'max_depth': max_depth
                    }

print("Best Parameters:", best_params, "Best Accuracy:", best_accuracy)

Parameters: learning_rate=0.1, n_estimators=100, subsample=1.0, max_depth=3
Accuracy: 0.99964

Parameters: learning_rate=0.1, n_estimators=100, subsample=1.0, max_depth=4
Accuracy: 1.0

Parameters: learning_rate=0.1, n_estimators=100, subsample=1.0, max_depth=5
Accuracy: 1.0

Parameters: learning_rate=0.1, n_estimators=100, subsample=0.8, max_depth=3
Accuracy: 0.99968

Parameters: learning_rate=0.1, n_estimators=100, subsample=0.8, max_depth=4
Accuracy: 1.0

Parameters: learning_rate=0.1, n_estimators=100, subsample=0.8, max_depth=5
Accuracy: 1.0

Parameters: learning_rate=0.1, n_estimators=100, subsample=0.5, max_depth=3
Accuracy: 1.0

Parameters: learning_rate=0.1, n_estimators=100, subsample=0.5, max_depth=4
Accuracy: 1.0

Parameters: learning_rate=0.1, n_estimators=100, subsample=0.5, max_depth=5
Accuracy: 1.0

Parameters: learning_rate=0.1, n_estimators=200, subsample=1.0, max_depth=3
Accuracy: 1.0

Parameters: learning_rate=0.1, n_estimators=200, subsample=1.0, max_depth=4
Accura

In [25]:
# Gradient Boosting

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score

# GB used tree regressors directly
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, subsample= 1.0, max_depth= 4, criterion='squared_error')
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test, y_pred)
f1_scorer = f1_score(y_test, y_pred, average='micro')
roc_auc = roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1], average='macro')

print("GradientBoosting Accuracy by Hold-out Evaluation: acc = ",acc, ', F1 = ',f1_scorer,', AUC = ',roc_auc)

GradientBoosting Accuracy by Hold-out Evaluation: acc =  1.0 , F1 =  1.0 , AUC =  1.0
